In [1]:
import pandas as pd
train=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/labeledTrainData.tsv",header=0,delimiter='\t',quoting=3)
test=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/testData.tsv",header=0,delimiter='\t',quoting=3)
unlabeled_train=pd.read_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/unlabeledTrainData.tsv",header=0,delimiter='\t',quoting=3)
print("Load %d labeled train data, %d unlabeled train data and %d test data" % (train["review"].size, unlabeled_train["review"].size, len(test['review'])))

Load 25000 labeled train data, 50000 unlabeled train data and 25000 test data


In [2]:
train.head(10)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."
5,"""8196_8""",1,"""I dont know why people think this is such a b..."
6,"""7166_2""",0,"""This movie could have been very good, but com..."
7,"""10633_1""",0,"""I watched this video at a friend's house. I'm..."
8,"""319_1""",0,"""A friend of mine bought this film for £1, and..."
9,"""8713_10""",1,"""<br /><br />This movie is full of references...."


In [3]:
#Don't remove stop words in Word2Vec
# After experiment, we should not use porter stemmer in Word2Vec approach!
negator=[]
with open("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/negative_words.txt") as f:
    for line in f.readlines():
        negator.append(line.strip())
#print negator

from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
def review_to_list(raw_review,remove_stopwords=False):
    #1. remove html
    raw_text=BeautifulSoup(raw_review,'lxml').get_text()
    #2. remove non-letter, replace n't (maybe we can hold the number)
    only_letter_text=re.sub("n\'t"," not",raw_text)
    only_letter_text=re.sub("[^a-zA-Z]"," ",only_letter_text)
    #3. split and lower case
    pure_text=only_letter_text.lower().split()
    #4. (optional) remove stop words
    if remove_stopwords:
        #stemmer=nltk.PorterStemmer()
        stopword=set(stopwords.words("english"))
        pure_text=[m for m in pure_text if m not in stopword or m in set(negator)]
    #5. return a list of words
    return pure_text

In [4]:
#Word2Vec input format: single sentences, each one as a list of words. [[list],[list]...]
tokenizer=nltk.data.load('tokenizers/punkt/english.pickle')
def review_to_sentences(review,tokenizer,remove_stopwords=False):
    sentences=[]
    raw_sentences=tokenizer.tokenize(review.strip())
    for item in raw_sentences:
        if(len(item)>0):
            sentences.append(review_to_list(item,remove_stopwords))
    return sentences

In [5]:
#word2Vec method
sentences=[]
print("Parsing sentences from training data...")
for review in train["review"]:
    sentences+=review_to_sentences(review.decode('utf-8'),tokenizer)
print("Parsing sentences from unlabeled training data...")
for review in unlabeled_train["review"]:
    sentences+=review_to_sentences(review.decode('utf-8'),tokenizer)
print len(sentences)

import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',level=logging.INFO)

num_features=100
min_word_count=1
num_workers=4
context=10
downsampling=1e-3

from gensim.models import word2vec
print "Training model..."
model=word2vec.Word2Vec(sentences,workers=num_workers,\
                        size=num_features,min_count=min_word_count,\
                        window=context,sample=downsampling)
model.init_sims(replace=True)
model_name="word2vec_100features_1minwords_10context"
model.save(model_name)

Parsing sentences from training data...


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: "." looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled training data...


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

795538


2016-11-03 22:26:40,589:INFO:collecting all words and their counts
2016-11-03 22:26:40,590:INFO:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-03 22:26:40,669:INFO:PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2016-11-03 22:26:40,747:INFO:PROGRESS: at sentence #20000, processed 451887 words, keeping 24948 word types


Training model...


2016-11-03 22:26:40,823:INFO:PROGRESS: at sentence #30000, processed 671310 words, keeping 30034 word types
2016-11-03 22:26:40,900:INFO:PROGRESS: at sentence #40000, processed 897810 words, keeping 34348 word types
2016-11-03 22:26:40,975:INFO:PROGRESS: at sentence #50000, processed 1116958 words, keeping 37761 word types
2016-11-03 22:26:41,046:INFO:PROGRESS: at sentence #60000, processed 1338399 words, keeping 40723 word types
2016-11-03 22:26:41,123:INFO:PROGRESS: at sentence #70000, processed 1561573 words, keeping 43333 word types
2016-11-03 22:26:41,197:INFO:PROGRESS: at sentence #80000, processed 1780880 words, keeping 45714 word types
2016-11-03 22:26:41,274:INFO:PROGRESS: at sentence #90000, processed 2004989 words, keeping 48135 word types
2016-11-03 22:26:41,349:INFO:PROGRESS: at sentence #100000, processed 2226945 words, keeping 50207 word types
2016-11-03 22:26:41,422:INFO:PROGRESS: at sentence #110000, processed 2446559 words, keeping 52081 word types
2016-11-03 22:26:41

In [13]:
from gensim.models import Word2Vec
model=Word2Vec.load("word2vec_5000features_40minwords_10context")
num_features=5000

In [9]:
model.doesnt_match("man child woman mother".split())

'man'

In [10]:
model.most_similar("iron")

[(u'ape', 0.6768560409545898),
 (u'spider', 0.6629838347434998),
 (u'eagle', 0.6501494646072388),
 (u'octopus', 0.6344279646873474),
 (u'cobra', 0.6295153498649597),
 (u'elephant', 0.6254611015319824),
 (u'tiger', 0.6231738328933716),
 (u'blade', 0.5876468420028687),
 (u'sword', 0.5871504545211792),
 (u'airplane', 0.5838525891304016)]

In [11]:
from gensim.models import Word2Vec
#model=Word2Vec.load("300features_40minwords_10context")
model.syn0.shape
#model["iron"]

(16490, 5000)

In [14]:
#word2Vec method
import numpy as np
def makeFeatureVec(words,model,numFeatures):
    res=np.zeros((numFeatures,),dtype='float32')
    count=0
    index2word_set=set(model.index2word)
    for item in words:
        if item in index2word_set:
            count+=1;
            res=np.add(res,model[item])
    res=np.divide(res,count)
    return res
def getAvgFeatureVecs(review,model,numFeatures):
    reviewFeatureVecs=np.zeros((len(review),numFeatures))
    count=0
    for item in review:
        if count%1000.==0.:
            print("Review %d of %d" % (count,len(review)))
        reviewFeatureVecs[count]=makeFeatureVec(item,model,numFeatures)
        count+=1
    return reviewFeatureVecs

clean_train_reviews=[]
for m in train["review"]:
    clean_train_reviews.append(review_to_list(m,remove_stopwords=True))
train_data_average=getAvgFeatureVecs(clean_train_reviews,model,num_features)
print("Creating review features vec for test data")
clean_test_reviews=[]
for m in test["review"]:
    clean_test_reviews.append(review_to_list(m,remove_stopwords=True))
test_data_average=getAvgFeatureVecs(clean_test_reviews,model,num_features)

Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating review features vec for test data
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of

In [117]:
from sklearn.ensemble import RandomForestClassifier
clf=RandomForestClassifier(n_estimators=100)
print("Fitting a random forest to labeled train data...")
forest=clf.fit(train_data_average,train["sentiment"])
result=forest.predict(test_data_average)
output=pd.DataFrame(data={"id":test['id'],"sentiment":result})
output.to_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/result_Word2Vec.csv",index=False,quoting=3)


Fitting a random forest to labeled train data...


In [19]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(class_weight="balanced")
logistic=clf.fit(train_data_average,train["sentiment"])
result=logistic.predict_proba(test_data_average)[:,1]
output=pd.DataFrame(data={"id":test['id'],"sentiment":result})
output.to_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/result_Word2Vec.csv",index=False,quoting=3)
#score of , word2vec, C default
#score of 0.928, word2vec, balanced class weight 5000features_40minwords_10context
#score of , word2vec, balanced class weight 100features_1minwords_10context

In [62]:
from sklearn.cluster import KMeans
import time
start=time.time()
wordVec=model.syn0
num_clusters=wordVec.shape[0]/5
kmeans_clustering=KMeans(n_clusters=num_clusters)
idx=kmeans_clustering.fit_predict(wordVec)
end=time.time()
elapse=end-start
print "Kmeans algorithm take time: ",elapse," seconds"

Kmeans algorithm take time:  1645.09027004  seconds


In [63]:
wordCentroidMap=dict(zip(model.index2word,idx))
def create_bag_of_centroids(wordList,wordCentroidMap):
    num_centroids=max(wordCentroidMap.values())+1
    bag_of_centroids=np.zeros(num_centroids,dtype="float32")
    for word in wordList:
        if word in wordCentroidMap:
            bag_of_centroids[wordCentroidMap[word]]+=1
    return bag_of_centroids    

In [66]:
train_centroid=np.zeros((train["review"].size,num_clusters),dtype="float32")
counter=0
for review in clean_train_reviews:
    train_centroid[counter]=create_bag_of_centroids(review,wordCentroidMap)
    counter+=1

test_centroid=np.zeros((test["review"].size,num_clusters),dtype="float32")
counter=0
for review in clean_test_reviews:
    test_centroid[counter]=create_bag_of_centroids(review,wordCentroidMap)
    counter+=1

In [67]:
forest = RandomForestClassifier(n_estimators = 100)

In [68]:
forest=forest.fit(train_centroid,train["sentiment"])
result2=forest.predict(test_centroid)
output=pd.DataFrame(data={"id":test["id"],"sentiment":result2})
output.to_csv("/Users/shichangtai/Desktop/Kaggle/Bag of Wprds Meet Bags of Popcorn/result3.csv",index=False,quoting=3)